In [1]:
!pip install -r requirements.txt

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install -e .

Obtaining file:///tf/notebooks/EPT/1_taxonomist
  Attempting uninstall: taxonomist
    Found existing installation: taxonomist 0.0.3
    Can't uninstall 'taxonomist'. No files were found to uninstall.
  Running setup.py develop for taxonomist
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
df = pd.read_csv('data/processed/ept-14/01_EPT_processed_5splits_Species_DNA.csv')
df.head()

,Specimen ID,Sample Name/Number,Species Name,Image File Name,Other Notes,Max Feret Diameter,Perimeter,Area,Holes,Area+Holes,...,Genus_DNA,Species_DNA,Label_DNA,read count_DNA,individual,0,1,2,3,4
0,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,1_4_F12_1_2023_12_14-09-51-53-989.PNG,C25_T_mit,352,923,25295,19,25314,...,Goera,Goera pilosa,Goera pilosa,215998.0,4_F12,test,train,train,val,train
1,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,2_4_F12_1_2023_12_14-09-51-54-000.PNG,C25_T_mit,367,1034,28245,168,28413,...,Goera,Goera pilosa,Goera pilosa,215998.0,4_F12,test,train,train,val,train
2,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,1_4_F12_1_2023_12_14-09-51-54-013.PNG,C25_T_mit,400,1093,28301,21,28322,...,Goera,Goera pilosa,Goera pilosa,215998.0,4_F12,test,train,train,val,train
3,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,2_4_F12_1_2023_12_14-09-51-54-025.PNG,C25_T_mit,380,999,29520,43,29563,...,Goera,Goera pilosa,Goera pilosa,215998.0,4_F12,test,train,train,val,train
4,4_F12,4_F12_2,PMR_ExStream22_Main_EPT,1_4_F12_2_2023_12_14-09-52-19-798.PNG,C25_T_mit,239,645,16395,1,16396,...,Goera,Goera pilosa,Goera pilosa,215998.0,4_F12,test,train,train,val,train


In [3]:
import os
import glob

def list_files_with_prefix(directory, prefix, model):
    # Define the pattern for the files to search
    pattern = os.path.join(directory, f"{prefix}*")
    
    # Use glob to find files matching the pattern
    files = glob.glob(pattern)
    
    return files

def list_ckpts(fold_directory):
    # Define the pattern for the files to search    
    pattern = os.path.join(fold_directory, '*.ckpt')
    
    # Use glob to find files matching the pattern, with recursive search
    ckpt_files = glob.glob(pattern)
    
    filtered_files = [f for f in ckpt_files if model in f if not f.endswith('_last.ckpt')]     # Filter out files ending with '_last.ckpt' and only return desired model
    
    # Return the first file from the filtered list or None if the list is empty
    return filtered_files

def extend_fold_dict(directory):
    for subfolder in directory:
        folds = [f.path for f in os.scandir(subfolder) if f.is_dir()]
        for fold in folds:
            fold_dict[int(fold[-1])].extend(list_ckpts(fold))      

In [4]:
# Searching for the best model checkpoint in the directory
directory = 'outputs/biodiscover/'
prefix = 'f02_EPT-14-biomass'
model = 'efficientnet_b0' # ResNet18 performance was worse for all taxonomic levels

    
fold_dict={}
for number in range(0,5):
    fold_dict[number] = []

# List the files
setting_folders = list_files_with_prefix(directory, prefix, model)

extend_fold_dict(setting_folders)

In [5]:
fold_dict

{0: ['outputs/biodiscover/f02_EPT-14-biomass-Order_DNA-efficientnet_b0_efficientnet_b0/f0/f02_EPT-14-biomass-Order_DNA-efficientnet_b0_efficientnet_b0_f0_240902-1647-7f6c_epoch17_val-loss0.03.ckpt',
  'outputs/biodiscover/f02_EPT-14-biomass-Species_DNA-efficientnet_b0_efficientnet_b0/f0/f02_EPT-14-biomass-Species_DNA-efficientnet_b0_efficientnet_b0_f0_240830-1701-5caa_epoch00_val-loss0.56.ckpt',
  'outputs/biodiscover/f02_EPT-14-biomass-Family_DNA-efficientnet_b0_efficientnet_b0/f0/f02_EPT-14-biomass-Family_DNA-efficientnet_b0_efficientnet_b0_f0_240903-1310-20be_epoch05_val-loss0.28.ckpt',
  'outputs/biodiscover/f02_EPT-14-biomass-Genus_DNA-efficientnet_b0_efficientnet_b0/f0/f02_EPT-14-biomass-Genus_DNA-efficientnet_b0_efficientnet_b0_f0_240904-1354-6be9_epoch15_val-loss0.56.ckpt'],
 1: ['outputs/biodiscover/f02_EPT-14-biomass-Order_DNA-efficientnet_b0_efficientnet_b0/f1/f02_EPT-14-biomass-Order_DNA-efficientnet_b0_efficientnet_b0_f1_240902-1833-d53a_epoch00_val-loss0.04.ckpt',
  'outp

In [1]:
import wandb
wandb.login() #581221dff0063a897a4eff9d285fa871a2c8e7f2

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: philipp-rehsen (uni-due). Use `wandb login --relogin` to force relogin


True

In [ ]:
# 01_ept-14-biomass was used to determine the learning rate for 'Log_weights'
# 02_ept-14-biomass was used to train for 100 epochs on 'Log_weights'
# 03_ept-14-biomass was used to pre-train for 100 epochs on 'Species_DNA', 'Genus_DNA', 'Familiy_DNA', 'Order_DNA'

# f'02_EPT-14-biomass-{label}-{model}' was used to pre-train for 100 epochs on 'Species_DNA', 'Genus_DNA', 'Familiy_DNA', 'Order_DNA' wit early stopping = True and 20 epochs patience

# Train biomass estimation on all 5 folds of previously created splits.
for fold, ls in fold_dict.items():
    for file in ls:
        if 'Species_DNA' in file:
            label = 'Species_DNA' # only used for naming, the train label is log_weights
        if 'Genus_DNA' in file:
            label = 'Genus_DNA' # only used for naming, the train label is log_weights
        if 'Family_DNA' in file:
            label = 'Family_DNA' # only used for naming, the train label is log_weights
        if 'Order_DNA' in file:
            label = 'Order_DNA' # only used for naming, the train label is log_weights
        !python scripts/02_train.py \
            --data_folder "../0_images/" \
            --dataset_config "conf/user_datasets.py" \
            --dataset_name "biodiscover" \
            --csv_path "data/processed/ept-14/01_EPT_processed_5splits_Species_DNA.csv" \
            --ckpt_path {file} \
            --label "Log_weights" \
            --fold {fold} \
            --class_map 'none' \
            --imsize 224 \
            --batch_size 256 \
            --aug 'EPT-biomass' \
            --load_to_memory 'False' \
            --model 'efficientnet_b0' \
            --freeze_base 'False' \
            --pretrained 'True' \
            --opt 'adamw' \
            --max_epochs 100 \
            --min_epochs 0 \
            --early_stopping 'True' \
            --early_stopping_patience 20 \
            --criterion 'l1' \
            --lr 0.001 \
            --auto_lr 'True' \
            --log_dir '03_EPT-14-biomass-pre-trained-unfrozen' \
            --out_folder 'outputs' \
            --out_prefix f'03_EPT-14-biomass-pre-trained-unfrozen-{label}' \
            --deterministic 'True' \
            --precision '16-mixed' \
            --resume 'False'

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Seed set to 42
/tf/notebooks/EPT/1_taxonomist/src/taxonomist/taxonomist_model.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start

Epoch 2: 100%|████████████████████| 355/355 [03:00<00:00,  1.96it/s, v_num=7867]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|████████████████████| 355/355 [03:00<00:00,  1.96it/s, v_num=7867]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  36%|██████▍           | 32/89 [00:09<00:16,  3.53it/s]


Epoch 4: 100%|████████████████████| 355/355 [02:59<00:00,  1.98it/s, v_num=7867]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|████████████████████| 355/355 [03:00<00:00,  1.97it/s, v_num=7867]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  45%|████████          | 40/89 [00:11<00:13,  3.62it/s]


Epoch 6: 100%|████████████████████| 355/355 [03:00<00:00,  1.97it/s, v_num=7867]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 7: 100%|████████████████████| 355/355 [03:00<00:00,  1.97it/s, v_num=7867]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  54%|█████████▋        | 48/89 [00:14<00:12,  3.40it/s]


Epoch 8: 100%|████████████████████| 355/355 [03:00<00:00,  1.97it/s, v_num=7867]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 9: 100%|████████████████████| 355/355 [02:58<00:00,  1.98it/s, v_num=7867]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  63%|███████████▎      | 56/89 [00:16<00:09,  3.44it/s]


...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 18: 100%|███████████████████| 355/355 [02:59<00:00,  1.98it/s, v_num=4810]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 19: 100%|███████████████████| 355/355 [02:59<00:00,  1.97it/s, v_num=4810]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 20: 100%|███████████████████| 355/355 [02:59<00:00,  1.98it/s, v_num=4810]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   4%|▊                  | 4/89 [00:00<00:08,  9.51it/s]


Epoch 21: 100%|███████████████████| 355/355 [02:59<00:00,  1.98it/s, v_num=4810]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 22: 100%|███████████████████| 355/355 [02:59<00:00,  1.98it/s, v_num=4810]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  13%|██▍               | 12/89 [00:02<00:18,  4.28it/s]


Epoch 22: 100%|███████████████████| 355/355 [03:28<00:00,  1.70it/s, v_num=4810]
/usr/local/lib/python3.8/dist-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:156: `.test(ckpt_path="best")` is called with Trainer configured with multiple `ModelCheckpoint` callbacks. It will use the best checkpoint path from first checkpoint callback.
Restoring states from the checkpoint path at /tf/notebooks/EPT/1_taxonomist/outputs/biodiscover/f03_EPT-14-biomass-pre-trained-unfrozen-Family_DNA_efficientnet_b0/f0/f03_EPT-14-biomass-pre-trained-unfrozen-Family_DNA_efficientnet_b0_f0_240910-1510-4810_epoch02_val-loss0.31.ckpt
/usr/local/lib/python3.8/dist-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/mai

Testing DataLoader 0: 100%|███████████████████| 111/111 [00:36<00:00,  3.08it/s]
────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────
     test/loss_epoch        0.4587896466255188
────────────────────────────────────────────────────────────────────────────────
Best model: /tf/notebooks/EPT/1_taxonomist/outputs/biodiscover/f03_EPT-14-biomass-pre-trained-unfrozen-Family_DNA_efficientnet_b0/f0/f03_EPT-14-biomass-pre-trained-unfrozen-Family_DNA_efficientnet_b0_f0_240910-1510-4810_epoch02_val-loss0.31.ckpt | score: 0.31387466192245483
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Seed set to 42
/tf/notebooks/EPT/1_taxonomist/src/taxonomist/taxonomist_model.py:23

In [1]:
import os
import glob

def list_files_with_prefix(directory, prefix):
    # Define the pattern for the files to search
    pattern = os.path.join(directory, f"{prefix}*")
    
    # Use glob to find files matching the pattern
    files = glob.glob(pattern)
    
    return files

def list_ckpts(fold_directory):
    # Define the pattern for the files to search    
    pattern = os.path.join(fold_directory, '*.ckpt')
    
    # Use glob to find files matching the pattern, with recursive search
    ckpt_files = glob.glob(pattern)
    
    filtered_files = [f for f in ckpt_files if not f.endswith('_last.ckpt')]     # Filter out files ending with '_last.ckpt'
    
    # Return the first file from the filtered list or None if the list is empty
    return filtered_files

def extend_fold_dict(directory):
    for subfolder in directory:
        folds = [f.path for f in os.scandir(subfolder) if f.is_dir()]
        for fold in folds:
            fold_dict[int(fold[-1])].extend(list_ckpts(fold))
        
directory = 'outputs/biodiscover/'
prefix = 'f03_EPT-14-biomass-pre-trained-unfrozen'

    
fold_dict={}
for number in range(0,5):
    fold_dict[number] = []

# List the files in fold_dict
setting_folders = list_files_with_prefix(directory, prefix)

extend_fold_dict(setting_folders)

In [2]:
for fold, ls in fold_dict.items():
    for file in ls:
        # Automatic predictiongs using the first (best) model checkpoint in the directory.
        !python scripts/03_predict.py \
            --data_folder "../0_images/" \
            --dataset_config "conf/user_datasets.py" \
            --dataset_name "biodiscover" \
            --csv_path "data/processed/ept-14/01_EPT_processed_5splits_Species_DNA.csv" \
            --label "Log_weights" \
            --fold {fold} \
            --class_map 'none' \
            --imsize 224 \
            --batch_size 512 \
            --aug 'EPT-biomass' \
            --out_folder 'outputs' \
            --tta 'False' \
            --out_prefix '03_EPT-14-biomass-pre-trained-unfrozen' \
            --ckpt_path {file}

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/tf/notebooks/EPT/1_taxonomist/src/taxonomist/taxonomist_model.py:503: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weigh

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/tf/notebooks/EPT/1_taxonomist/src/taxonomist/taxonomist_model.py:503: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weigh

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/tf/notebooks/EPT/1_taxonomist/src/taxonomist/taxonomist_model.py:503: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weigh

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/tf/notebooks/EPT/1_taxonomist/src/taxonomist/taxonomist_model.py:503: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weigh

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/tf/notebooks/EPT/1_taxonomist/src/taxonomist/taxonomist_model.py:503: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weigh

In [7]:
import os
import glob


def list_files_with_prefix(directory, prefix):
    pattern = os.path.join(directory, f"{prefix}*")
    files = glob.glob(pattern)
    
    return files

def list_csvs(directory): 
    pattern = os.path.join(directory, 'predictions', 'biodiscover_EPT-biomass', '*.csv')
    csv_files = glob.glob(pattern)
    return csv_files

def extend_run_dict_csv(directory):
    folds = [f.path for f in os.scandir(directory) if f.is_dir()]
    for fold in folds:
        run_dict_csv[directory].extend(list_csvs(fold))
        
directory = 'outputs/biodiscover/'
prefix = 'f03_EPT-14-biomass-pre-trained-unfrozen'        
        
run_dict_csv={}
for run in list_files_with_prefix(directory, prefix):
    run_dict_csv[run] = []
    
setting_folders = list_files_with_prefix(directory, prefix)

for setting in setting_folders:
    extend_run_dict_csv(setting)

In [8]:
run_dict_csv

{'outputs/biodiscover/f03_EPT-14-biomass-pre-trained-unfrozen-Species_DNA_efficientnet_b0': ['outputs/biodiscover/f03_EPT-14-biomass-pre-trained-unfrozen-Species_DNA_efficientnet_b0/f0/predictions/biodiscover_EPT-biomass/03_EPT-14-biomass-pre-trained-unfrozen_f03_EPT-14-biomass-pre-trained-unfrozen-Species_DNA_efficientnet_b0_f0_240910-1349-38ba_epoch01_val-loss0.33_EPT-biomass.csv',
  'outputs/biodiscover/f03_EPT-14-biomass-pre-trained-unfrozen-Species_DNA_efficientnet_b0/f1/predictions/biodiscover_EPT-biomass/03_EPT-14-biomass-pre-trained-unfrozen_f03_EPT-14-biomass-pre-trained-unfrozen-Species_DNA_efficientnet_b0_f1_240910-1937-b7e0_epoch05_val-loss0.45_EPT-biomass.csv',
  'outputs/biodiscover/f03_EPT-14-biomass-pre-trained-unfrozen-Species_DNA_efficientnet_b0/f3/predictions/biodiscover_EPT-biomass/03_EPT-14-biomass-pre-trained-unfrozen_f03_EPT-14-biomass-pre-trained-unfrozen-Species_DNA_efficientnet_b0_f3_240911-1035-063b_epoch29_val-loss0.49_EPT-biomass.csv',
  'outputs/biodiscove

In [9]:
import pandas as pd
from pathlib import Path

ref = pd.read_csv("data/processed/ept-14/01_EPT_processed_5splits_Species_DNA.csv")

for run, fold_ls in run_dict_csv.items():
    print('joining '+ run)
    df_f0 = pd.read_csv(fold_ls[0])
    df_f1 = pd.read_csv(fold_ls[1])
    df_f2 = pd.read_csv(fold_ls[2])
    df_f3 = pd.read_csv(fold_ls[3])
    df_f4 = pd.read_csv(fold_ls[4])
    df = pd.concat([df_f0, df_f1, df_f2, df_f3, df_f4])

    if 'Species_DNA' in run:
        label = 'Species_DNA' # only used for naming, the train label is log_weights
    if 'Genus_DNA' in run:
        label = 'Genus_DNA' # only used for naming, the train label is log_weights
    if 'Family_DNA' in run:
        label = 'Family_DNA' # only used for naming, the train label is log_weights
    if 'Order_DNA' in run:
        label = 'Order_DNA' # only used for naming, the train label is log_weights
        
    # Get just the filename from the full path
    df["fname"] = df["fname"].apply(lambda x: Path(x).name)
    preds = df[["fname", "y_pred"]].set_index("fname")

    # Join the dataframes. 'inner' to keep only the rows that are in both dataframes
    results_df = ref.join(preds, how="inner", on="Image File Name")  

    # Choose wanted columns
    results_df = results_df[['Log_weights', "y_pred", "individual"]]

    # Grouped by individual
    results_df_grouped = results_df.groupby(["individual"]).apply(lambda group: group[(group >= group.quantile(0.05)) & (group <= group.quantile(0.95))].mean())

    results_df_grouped.to_csv(f'{run}/03_EPT-14-biomass-pre-trained-unfrozen_{label}.csv')    

joining outputs/biodiscover/f03_EPT-14-biomass-pre-trained-unfrozen-Species_DNA_efficientnet_b0
joining outputs/biodiscover/f03_EPT-14-biomass-pre-trained-unfrozen-Genus_DNA_efficientnet_b0
joining outputs/biodiscover/f03_EPT-14-biomass-pre-trained-unfrozen-Family_DNA_efficientnet_b0
joining outputs/biodiscover/f03_EPT-14-biomass-pre-trained-unfrozen-Order_DNA_efficientnet_b0


In [ ]:
## Until here, the code works fine. The following parts are still more or less experimental and for sure not cleaned up.

In [ ]:
import pandas as pd
confusion_df = pd.read_csv('outputs/biodiscover/02_ept-14-biomass_efficientnet_b0/01_EPT_processed_5splits_Species_DNA_grouped_predictions.csv')

confusion_matrixX(confusion_df['y_true'], confusion_df['y_pred'], 'union')